# IMPORT

In [1]:
try:
    import google.colab # type: ignore
    colab = True
except:
    colab = False

if colab:
    !git clone "https://github.com/cybernetic-m/eai-project.git" # type: ignore

In [2]:
# Model
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim
from torch.optim.lr_scheduler import ExponentialLR 

# Others
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import json
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pickle
 
# Our files
if colab:
    sys.path.append('/content/eai-project/training')
    sys.path.append('/content/eai-project/preprocessing')
    sys.path.append('/content/eai-project/dataset')
    sys.path.append('/content/eai-project/utils')
    sys.path.append('/content/eai-project/models')
    sys.path.append('/content/eai-project/testing')
    from train import train
    from preprocessing import *
    from thermal_dataset import thermal_dataset 
    from utils import *
    from complete_model import complete_model 
    from lstm_only import lstm_only
    from testing import test
    prefix = '/content'
        
else:
    from training.train import train
    from preprocessing.preprocessing import *
    from dataset.thermal_dataset import thermal_dataset
    from utils.csv_utils import *
    from models.complete_model import complete_model
    from models.complete_model import complete_model_autoencoder
    from models.lstm_only import lstm_only
    from testing.test import test
    from blocks import mlp, linear, rnn, lstm
    prefix = '.'
    
pd.set_option('display.max_columns', None)

In [3]:
hyper_parameters = {'num_epochs': 100,
                    'batch_size': 256,
                    'hidden_dim' : 3,
                    'lr': 0.001,
                    'mode': 'auto-weighted', #average, median, auto-weighted
                    'extractor_type': 'lstm_autoencoder', #lstm_autoencoder (if you want to use the lstm autoencoder), conv (if you want to use the convolutional autoencoder) or lstm_encoder (if you want to use only the lstm encoder)
                    'ensemble': True,
                    'timesteps': 200,
                    'window_size':30,
                    'norm': 'Std', # Not (Raw Data), Minmax (MinMax Scaling), Std (Standard Scaling)
                    'file':1,
                    'weight_decay': 0,
                    'pretrain':False,
                    'autoencoder_train' : True, # to train only the autoencoder
                    'heterogeneous': True,
                    'lr_multipliers_extractor': 1,
                    'lr_multipliers_ensemble': {
                        mlp: 1,
                        linear: 1,
                        rnn: 1,
                        lstm: 1,
                    }

                        }

# Parameters of the convolutional autoencoder
conv_autoencoder_dict = {'in_kern_out': [[4, 3, 5]], # List of hyperparam of autoencoder [[in_channels, kernel_size, out_channels], ...]
                    'pooling_kernel_size': 2, # how much big is the kernel of the pooling (i.e. 2 means halving the dimension each layer)
                    'padding': 'same', # 'same', 'full', 'valid'
                    'pooling': 'avg', # 'max' for Max Pooling and 'avg' for Average Pooling 
                    'scale_factor': 2, # upsample scale_factor, 2 means double the dimension each layer
                    'upsample_mode': 'linear', # mode of the upsampling 'linear', 'nearest'
                    'dropout': 0.0
 }

lstm_autoencoder_dict = {'in_hidd': [[4, 2]], # List of hyperparam of autoencoder [[input_dim, hidden_dim], ...]
                        'dropout': 0.3,
                        'lstm_layers':1,
 }


if hyper_parameters['extractor_type'] == 'conv':
    feature_dim = int(( hyper_parameters['timesteps'] / (2**(len(conv_autoencoder_dict['in_kern_out'])-1))*conv_autoencoder_dict['in_kern_out'][-1][-1]) /2)
    autoencoder_dict = conv_autoencoder_dict
elif hyper_parameters['extractor_type'] == 'lstm_autoencoder' or hyper_parameters['extractor_type'] == 'lstm_encoder':
    feature_dim = hyper_parameters['timesteps']*lstm_autoencoder_dict['in_hidd'][-1][-1] 
    autoencoder_dict = lstm_autoencoder_dict
if hyper_parameters['heterogeneous']:
    feature_dim += 3
    

# Definition of the model (You can decomment to include more models)
if hyper_parameters['autoencoder_train'] == False:
    ensemble_model = {
        'mlp': [{'layer_dim_list': [ feature_dim,int(feature_dim/1.5),int(feature_dim/3),int(feature_dim/5),int(feature_dim/8),int(feature_dim/10),3]}],
                #{'layer_dim_list': [ feature_dim,int(feature_dim*1.5),3]}], 
        #'ARIMA': [{'p': 2, 'd': 0, 'q': 2, 'ps': 0, 'ds': 0, 'qs': 0, 's': 1}], 
        'linear_regressor': [{'in_features': feature_dim, 'out_features': 3, 'bias':False}],
        'lstm': [{'feature_dim':3, 'input_dim':feature_dim, 'out_features': 3, 'bias':False, 'num_layers':2}],
        #'rnn': [{'feature_dim':3, 'input_dim':feature_dim, 'out_features': 3, 'bias':False, 'num_layers':2}],
    }

else:
    ensemble_model = {} # Set the dictionary of the ensemble model to void if we want to train only the autoencoder

In [4]:
skip = False

if os.path.exists(prefix+'/data/X'+'training'+str(hyper_parameters['file'])+'.npy') and os.path.exists(prefix+'/data/Y'+str(hyper_parameters['window_size'])+'training'+str(hyper_parameters['file'])+'.npy'):
    print("Data already exists!")
    print(prefix+'/data/X'+'training'+str(hyper_parameters['file'])+'.npy')
    print(prefix+'/data/Y'+str(hyper_parameters['window_size'])+'training'+str(hyper_parameters['file'])+'.npy')
    skip = True

Data already exists!
./data/Xtraining1.npy
./data/Y30training1.npy


**Reproducibility and Device Setting**

In [5]:
# Set a seed for reproducibility purposes
seed = 46
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

# Set the device (cuda for Nvidia GPUs, mps for M1, M2 .. Apple Silicon)
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

# DATASET

In [6]:
link_zipped_csv = 'https://drive.google.com/file/d/1MssQF4pI_rZqiiDBP4XaLTT1ZaN6ykLm/view?usp=drive_link'
gdrive_link = 'https://drive.google.com/uc?id='
csv_dir = './csv'
zipped_file = './csv.zip'

download_csv(
    link_zipped_csv,
    gdrive_link,
    zipped_file
)

unzip_csv(
    zipped_file,
    csv_dir,
)


CSV file already downloaded!
CSV file already unzipped!


In [7]:
if not skip:

    path = '/content/csv/thermal_drift_features_lab_05_02.csv'

    # Read all the CSV files containing the Temperatures
    features_1 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_02.csv'))
    features_2 = pd.read_csv(os.path.join(prefix, 'csv/thermal_drift_features_lab_05_03.csv'))
    features_3 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_04.csv'))
    features_4 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_05.csv'))
    features_5 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_features_lab_05_06.csv'))

    # Read all the CSV files containing the X1, Y1, Z1 
    targets_1 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_02.csv'))
    targets_2 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_03.csv'))
    targets_3 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_04.csv'))
    targets_4 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_05.csv'))
    targets_5 = pd.read_csv(os.path.join(prefix,'csv/thermal_drift_targets_lab_05_06.csv'))

    features = [features_1, features_2, features_3, features_4, features_5]
    targets = [targets_1,targets_2,targets_3,targets_4,targets_5]

In [8]:
if not skip:
    for feature, target in zip(features, targets):
        feature.drop([
            "name", "tags",
            "2\"Tray1 Vacuum Sensor", "2\"Tray2 Vacuum Sensor", "2\"Tray3 Vacuum Sensor",
            "Avg Oven Temperature", "Chuck Temp [Cdeg]", "Chuck Temp2 [Cdeg]",
            "Chuck1 Vacuum Sensor", "Contrast", "Device State",
            "Dispenser1 Pressure Sensor", "Machine Room Temp", "Main Air", "Main Vacuum",
            "Oven Temperature", "PE_Rx", "PE_Ry", "PE_Rz", "PE_X1", "PE_Y1", "PE_Z1",
            "PUT1 Flow Sensor", "PUT2 Flow Sensor1", "PUT2 Flow Sensor2",
            "PUT2 Flow Sensor3", "PUT2 Flow Sensor4", "PUT2 Flow Sensor5",
            "Photodiode", "Pixel Power", "Preciser1 Vacuum Sensor",
            "Tec FIB1 Holder", "Tec FIB1 Plate", "Tec FIB2 Holder", "Tec FIB2 Plate",
            "Torque11","Torque2","Torque3","Torque4","Torque5","Torque6"
        ], axis=1, inplace=True)
        if 'name' in target.keys() and 'tags' in target.keys():

            target.drop(['name', 'tags'], axis=1, inplace=True)
            
            


In [9]:
if not skip:
    print(features[1]) # Print the features_1 table

In [10]:
if not skip:

    print(targets[1]) # Print the target_1

In [11]:
if not skip:
    # Put X1, Y1, Z1 on the same row of X1 eliminating the NAN values
    fixed_targets = [] # Create a list of target in which we put X1, Y1, Z1 in the same row
    for target in targets:
        fixed_targets.append(transform_dataframe(target)) # iterate over target_1,2,3 ... and append in fixed_targets
        
        

In [12]:
if not skip:

    print(fixed_targets[1]) # Print the fixed_target_1

In [13]:
if not skip:
    # Merge of targets with features in one single dataframe
    complete_numbers_list = [] # List of the table with columns that are numbers (0,1,2..) in which we unify both features and targets merging on closest time row
    for fixed_target, feature in zip(fixed_targets, features):
        complete_numbers_list.append(merge_on_closest_time(fixed_target.reset_index(), feature.reset_index()))

    trainig_number_list = []
    testing_number_list = []
    for i in range(len(complete_numbers_list)):
        part_numbers_list = complete_numbers_list[:i] + complete_numbers_list[i+1:]
        trainig_number_list.append(pd.concat(part_numbers_list))
        testing_number_list.append(complete_numbers_list[i])
        
    complete_numbers_dataframe = pd.concat(complete_numbers_list)
    


In [14]:
if not skip: 
    print(complete_numbers_list) # Print of one example

In [15]:
if not skip:

    complete = complete_numbers_dataframe.rename(columns={
        0: 'id',
        1: 'time',
        2: 'X1',
        3: 'Y1',
        4: 'Z1',
        5: 'to_remove',
        6: 'time_2',
        7: 'Temp1',
        8: 'Temp2',
        9: 'Temp3',
        10: 'Temp4'
        })
    complete.drop(['time', 'to_remove', 'time_2'], axis=1, inplace=True)
    training_list = []
    testing_list = []
    for training, testing in zip(trainig_number_list, testing_number_list):
        training_tmp = training.rename(columns={
            0: 'id',
            1: 'time',
            2: 'X1',
            3: 'Y1',
            4: 'Z1',
            5: 'to_remove',
            6: 'time_2',
            7: 'Temp1',
            8: 'Temp2',
            9: 'Temp3',
            10: 'Temp4'
            })
        training_tmp.drop(['time', 'to_remove', 'time_2'], axis=1, inplace=True)
        training_list.append(training_tmp)
        
        testing_tmp = testing.rename(columns={
            0: 'id',
            1: 'time',
            2: 'X1',
            3: 'Y1',
            4: 'Z1',
            5: 'to_remove',
            6: 'time_2',
            7: 'Temp1',
            8: 'Temp2',
            9: 'Temp3',
            10: 'Temp4'
            })
        testing_tmp.drop(['time', 'to_remove', 'time_2'], axis=1, inplace=True)
        testing_list.append(testing_tmp)
        

In [16]:
if not skip:

    print(training_list)

In [17]:

if not skip:
    
    # Save the features and targets in file npy
    for i, (testing, training) in enumerate(zip(testing_list, training_list)):

        # Transform the training and test data in float
        training.astype(float) 
        testing.astype(float) 

        # Take from dataframe the values of the columns of temperatures and positions saving into smallest dataframe of training/test
        X_train = training[['Temp1','Temp2', 'Temp3', 'Temp4']] 
        Y_train = training[['X1', 'Y1', 'Z1']] 
        X_test = testing[['Temp1','Temp2', 'Temp3', 'Temp4']] 
        Y_test = testing[['X1', 'Y1', 'Z1']] 

        # Transform the X, Y from dataframe in numpy array both for test and train
        X_train = X_train.values.astype(np.float32) 
        Y_train = Y_train.values.astype(np.float32) 
        X_test = X_test.values.astype(np.float32) 
        Y_test = Y_test.values.astype(np.float32) 

        # Do the gradient of the positions both for test and train
        Y_train = my_gradient(Y_train, window_size=hyper_parameters['window_size']) 
        Y_test = my_gradient(Y_test, window_size=hyper_parameters['window_size']) 
 
        np.save(prefix+'/data'+'/X'+'training'+str(i)+'.npy',X_train)  
        np.save(prefix+'/data'+'/Y'+str(hyper_parameters['window_size'])+'training'+str(i)+'.npy',Y_train)
        np.save(prefix+'/data'+'/X'+'testing'+str(i)+'.npy',X_test)
        np.save(prefix+'/data'+'/Y'+str(hyper_parameters['window_size'])+'testing'+str(i)+'.npy',Y_test)

In [18]:
if not skip:

    targets_1['time'] = pd.to_datetime(targets_1['time'], unit='ns')

    # Reshape the DataFrame using melt()
    targets_melted = targets_1.reset_index().melt(id_vars=['time'], value_vars=['X1', 'Y1', 'Z1'], var_name='variable', value_name='value')

    # Drop rows where 'value' is NaN (to keep only the non-null entries)
    targets_melted = targets_melted.dropna(subset=['value'])

    # Plot the data
    plt.figure(figsize=(10, 6))
    for var in targets_melted['variable'].unique():
        # Filter data for each variable and plot
        temp_df = targets_melted[targets_melted['variable'] == var]
        plt.plot(temp_df['time'], temp_df['value'], label=var)

    # Add labels and title
    plt.xlabel('Time')
    plt.ylabel('Values')
    plt.title('Plot of X1, Y1, Z1 over Time')
    plt.legend(title='Variable')

    # Show the plot
    plt.show()

    #targets.plot(y='X1',x='time')

In [19]:
if not skip:

    # Convert 'time' to datetime (nanoseconds to datetime)
    features_1['time'] = pd.to_datetime(features_1['time'], unit='ns')

    # Set 'time' as the index
    features_1.set_index('time', inplace=True)

    # Optionally, you can plot X1, Y1, Z1 directly
    features_1[['Temp Sensor 1', 'Temp Sensor 2', 'Temp Sensor 3', 'Temp Sensor 4', ]].plot()

    # Add labels and title
    plt.xlabel('Time')
    plt.ylabel('Values')
    plt.title('Plot of Temp Sensor 1, Temp Sensor 2, Temp Sensor 3, Temp Sensor 4 over Time')

    # Show the plot
    plt.show()

## Preprocessing

In [20]:
X = np.load(prefix+'/data'+'/X'+'training'+str(hyper_parameters['file'])+'.npy') # Training data
Y = np.load(prefix+'/data'+'/Y'+str(hyper_parameters['window_size'])+'training'+str(hyper_parameters['file'])+'.npy')

mean_X, std_X, min_val_X, max_val_X = torch.tensor([1.0, 1.0, 1.0, 1.0]), torch.tensor([0.0, 0.0, 0.0, 0.0]), torch.tensor([-1.0, -1.0, -1.0, -1.0]), torch.tensor([1.0,1.0, 1.0, 1.0]) # Initialization of values
mean_Y, std_Y, min_val_Y, max_val_Y = torch.tensor([1.0,1.0,1.0]), torch.tensor([0.0,0.0,0.0]), torch.tensor([-1.0,-1.0,-1.0]), torch.tensor([1.0,1.0,1.0]) # Initialization of values

#hyper_parameters['norm'] = 'Minmax'
# Fit the scaling to understand the parameters of the training set
if hyper_parameters['norm'] != 'Not':
    if hyper_parameters['norm'] == 'Minmax':
        scaler_X = MinMaxScaler()
        scaler_Y = MinMaxScaler()
        scaler_X.fit(X) # Fit only on training data
        scaler_Y.fit(Y) # Fit only on training data
        min_val_X, max_val_X = torch.tensor(scaler_X.data_min_.astype(np.float32)).to(device), torch.tensor(scaler_X.data_max_.astype(np.float32)).to(device) # Get minimum and maximum values
        min_val_Y, max_val_Y = torch.tensor(scaler_Y.data_min_.astype(np.float32)).to(device), torch.tensor(scaler_Y.data_max_.astype(np.float32)).to(device) # Get minimum and maximum values
        print("Min Value [temp1, temp2, temp3, temp4]:", min_val_X[0].item(),min_val_X[1].item(), min_val_X[2].item(), min_val_X[3].item(), "\nMax Value [temp1, temp2, temp3, temp4]:", max_val_X[0].item(), max_val_X[1].item(), max_val_X[2].item(), max_val_X[3].item())
        print("Min Value [X1, Y1, Z1]:", min_val_Y[0].item(),min_val_Y[1].item(), min_val_Y[2].item(), "\nMax Value [X1, Y1, Z1]:", max_val_Y[0].item(), max_val_Y[1].item(), max_val_Y[2].item())

    elif hyper_parameters['norm'] == 'Std':
        scaler_X = StandardScaler()
        scaler_Y = StandardScaler()
        scaler_X.fit(X)  # Fit only on training data
        scaler_Y.fit(Y)  # Fit only on training data
        mean_X, std_X = torch.tensor(scaler_X.mean_.astype(np.float32)).to(device), torch.tensor(scaler_X.scale_.astype(np.float32)).to(device) # Get mean & std
        mean_Y, std_Y = torch.tensor(scaler_Y.mean_.astype(np.float32)).to(device), torch.tensor(scaler_Y.scale_.astype(np.float32)).to(device) # Get mean & std
        print("Mean Value [temp1, temp2, temp3, temp4]:", mean_X[0].item(),mean_X[1].item(), mean_X[2].item(), mean_X[3].item(), "\nStandard Deviation [temp1, temp2, temp3, temp4]:", std_X[0].item(), std_X[1].item(), std_X[2].item(), std_X[3].item())
        print("Mean Value [X1, Y1, Z1]:", mean_Y[0].item(),mean_Y[1].item(), mean_Y[2].item(), "\nStandard Deviation  [X1, Y1, Z1]:", std_Y[0].item(), std_Y[1].item(), std_Y[1].item())
else:
    print("Using Raw Data!")

#print(X)
#print(Y)

splitPerc = [0.7,0.3]
splitted_X = split(X, splitPerc)
splitted_Y = split(Y, splitPerc)

Mean Value [temp1, temp2, temp3, temp4]: 2117.257568359375 2155.469970703125 2150.99951171875 2157.142578125 
Standard Deviation [temp1, temp2, temp3, temp4]: 22.64933204650879 23.240192413330078 21.471412658691406 20.387102127075195
Mean Value [X1, Y1, Z1]: 5.152535595698282e-05 3.1649022275814787e-05 -1.0432709984797839e-07 
Standard Deviation  [X1, Y1, Z1]: 0.0024545718915760517 0.009821190498769283 0.009821190498769283


# MODEL

In [21]:
datasetTrain = thermal_dataset((splitted_X[0],splitted_Y[0]), hyper_parameters['timesteps'], device)
datasetVal = thermal_dataset((splitted_X[1],splitted_Y[1]), hyper_parameters['timesteps'], device)

In [22]:
# Training, Test and Validation Dataloader initialization
loaderTrain = DataLoader(datasetTrain, shuffle=True, batch_size=hyper_parameters['batch_size'])
loaderVal = DataLoader(datasetVal, shuffle=True, batch_size=hyper_parameters['batch_size'])

# Model Initialization (True if you want to use the Ensemble model, False in you want to use a single LSTM model)
if hyper_parameters['ensemble'] == False:
    model = lstm_only(hidden_dim=hyper_parameters['hidden_dim'],
                      timesteps=hyper_parameters['timesteps'] ,
                      input_dim=4, 
                      output_dim=3).to(device)
    
elif hyper_parameters['ensemble'] == True:
    if hyper_parameters['extractor_type'] == 'conv':
        model = complete_model_autoencoder(ensemble_model, 
                                           device, 
                                           autoencoder_dim=conv_autoencoder_dict['in_kern_out'], 
                                           pooling_kernel_size=conv_autoencoder_dict['pooling_kernel_size'], 
                                           padding=conv_autoencoder_dict['padding'], 
                                           pooling=conv_autoencoder_dict['pooling'], 
                                           scale_factor = conv_autoencoder_dict['scale_factor'], 
                                           upsample_mode=conv_autoencoder_dict['upsample_mode'], 
                                           dropout=conv_autoencoder_dict['dropout'],
                                           mode=hyper_parameters['mode'],
                                           heterogeneous=hyper_parameters['heterogeneous'],
                                           timesteps=hyper_parameters['timesteps'],
                                           norm=hyper_parameters['norm'],
                                           mean=(mean_X, mean_Y),
                                           std=(std_X, std_Y),
                                           min_val= (min_val_X, min_val_Y),
                                           max_val= (max_val_X, max_val_Y),
                                           ).to(device)
    
    elif hyper_parameters['extractor_type'] == 'lstm_autoencoder' or hyper_parameters['extractor_type'] == 'lstm_autoencoder':
        model = complete_model(
                                    model_dict=ensemble_model, 
                                    device=device, 
                                    timesteps=hyper_parameters['timesteps'],
                                    lstm_layers=lstm_autoencoder_dict['lstm_layers'],
                                    autoencoder_dim=lstm_autoencoder_dict['in_hidd'], 
                                    dropout=lstm_autoencoder_dict['dropout'],
                                    extractor_type=hyper_parameters['extractor_type'],
                                    heterogeneous=hyper_parameters['heterogeneous'],
                                    norm=hyper_parameters['norm'],
                                    mode=hyper_parameters['mode'],
                                    mean=(mean_X, mean_Y),
                                    std=(std_X, std_Y),
                                    min_val= (min_val_X, min_val_Y),
                                    max_val= (max_val_X, max_val_Y)
                                    ).to(device)

# Definition of the optimizer and loss function
if hyper_parameters['ensemble'] == False:
    optimizer = optim.Adam(model.parameters(), lr=hyper_parameters['lr'])
elif hyper_parameters['ensemble'] == True:


    models, arima, rnn_ = model.get_models()
    list_models = models + arima + rnn_ 
    optimizer = [optim.Adam(model.extractor.parameters(), lr=hyper_parameters['lr']*hyper_parameters['lr_multipliers_extractor'])]
    for sub_model in list_models: 
        for model_type, multiplier in hyper_parameters['lr_multipliers_ensemble'].items():
            if isinstance(sub_model, model_type):
                optimizer.append(optim.Adam(model.parameters(), lr=hyper_parameters['lr']*multiplier, weight_decay=hyper_parameters['weight_decay']))
                break
    
hyper_parameters['lr_multipliers_ensemble'] = {
        'mlp': hyper_parameters['lr_multipliers_ensemble'][mlp],
        'linear': hyper_parameters['lr_multipliers_ensemble'][linear],
        'rnn': hyper_parameters['lr_multipliers_ensemble'][rnn],
        'lstm': hyper_parameters['lr_multipliers_ensemble'][lstm],
    }


loss_fn = nn.MSELoss()
if hyper_parameters['ensemble'] == False:
    scheduler = ExponentialLR(optimizer, gamma=0.85)
elif hyper_parameters['ensemble'] == True:
    scheduler = [ExponentialLR(opti, gamma=0.85) for opti in optimizer]


Autoencoder type: LSTM
Autoencoder Summary: lstm_autoencoder(
  (encoder): lstm_encoder(
    (lstm_layers): ModuleList(
      (0): lstm(
        (lstm): LSTM(4, 2, batch_first=True, dropout=0.3)
      )
    )
  )
  (decoder): lstm_decoder(
    (lstm_layers): ModuleList(
      (0): lstm(
        (lstm): LSTM(2, 4, batch_first=True, dropout=0.3)
      )
    )
    (linear_decoder): Linear(in_features=800, out_features=800, bias=True)
  )
)


/Users/cyber_m/Desktop/EAI-1/eai-project/.venv/lib/python3.9/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn(


In [23]:
if hyper_parameters['pretrain']:
    model.load('./results/training_2025-02-25_17-35/autoencoder.pt',autoencoder=hyper_parameters['pretrain'])

train(
    num_epochs=hyper_parameters['num_epochs'],
    loss_fn=loss_fn,
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    training_dataloader=loaderTrain,
    validation_dataloader=loaderVal,
    hyperparams=hyper_parameters,
    model_dict = ensemble_model,
    autoencoder_dict=autoencoder_dict,
    complete=hyper_parameters['ensemble'],
    autoencoder_train=hyper_parameters['autoencoder_train']
)


EPOCH 1/100:
Model saved to results/training_2025-03-06_15-09/autoencoder.pt
train AUTOENCODER: LOSS 143.973072298111 MAE temp1:8.0915, temp2:8.7283, temp3:8.3931, temp4:6.9962 R2 temp1:0.7525, temp2:0.7244, temp3:0.7229, temp4:0.7430 RMSE temp1:11.623901, temp2:12.551869, temp3:11.688844, temp4:8.909803
valid AUTOENCODER: LOSS 76.973572217501 MAE temp1:5.8415, temp2:5.9044, temp3:6.5321, temp4:6.9962 R2 temp1:0.8154, temp2:0.8169, temp3:0.7818, temp4:0.7430 RMSE temp1:8.380233, temp2:9.140293, temp3:8.571877, temp4:8.909803
EPOCH 2/100:
Model saved to results/training_2025-03-06_15-09/autoencoder.pt
train AUTOENCODER: LOSS 45.332289049702 MAE temp1:4.0325, temp2:5.4573, temp3:4.5811, temp4:5.1984 R2 temp1:0.9525, temp2:0.8996, temp3:0.9274, temp4:0.8562 RMSE temp1:5.050875, temp2:7.570094, temp3:5.924660, temp4:6.662415
valid AUTOENCODER: LOSS 48.104640813974 MAE temp1:4.5178, temp2:5.4546, temp3:4.8270, temp4:5.1984 R2 temp1:0.8906, temp2:0.8585, temp3:0.8776, temp4:0.8562 RMSE temp1